In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

from datetime import datetime
from datetime import date

In [132]:
df_model = pd.read_csv("../inter_data/preprocessed.csv")

In [133]:
df_model = df_model[[col for col in df_model.columns if "Unnamed" not in col]]

In [134]:
df_model = df_model.fillna(0)

In [135]:
df_model

,cm_cd,cm_mobile,refer_count,activation_count,IncomePA,cm_occup,E2Channel,IsKarvy,KRA YN,age,months_since_last_login,months_since_last_trade,total_logins_one_year,total_trades_one_year,referees_avg_logins_in_last_year,referees_avg_trades_in_last_year,num_referees_logging_in,num_referees_trading
0,76556800,6000015546,1,0.0,5_10L,P,CAT,0,N,22.0,1.791433,132.000000,10.0,0.0,0.000000,0.000000,0,0
1,49282624,6000016229,1,0.0,<=1L,S,CAT,0,Y,22.0,0.838641,1.528594,35.0,1.0,0.000000,0.000000,0,0
2,60000270,6000027031,1,0.0,<=1L,P,CAT,0,N,22.0,1.134335,1.068625,4.0,1.0,0.000000,0.000000,0,0
3,64606053,6000031654,31,23.0,<=1L,S,CAT,0,N,32.0,0.312963,3.171338,208.0,5.0,3.833333,1.608696,23,21
4,86330038,6000037524,3,0.0,<=1L,S,CAT,0,Y,23.0,1.035770,2.645660,69.0,1.0,0.000000,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33381,99998155,9999815503,1,0.0,<=1L,B,CAT,0,N,27.0,132.000000,132.000000,0.0,0.0,0.000000,0.000000,0,0
33382,INDERSI5,9999888891,1,0.0,1_5L,0,Premia,0,N,36.0,0.312963,0.542947,1367.0,292.0,0.000000,0.000000,0,0
33383,59684125,9999893553,1,1.0,<=1L,P,CAT,0,N,25.0,0.312963,0.542947,1635.0,63.0,477.000000,68.000000,1,1
33384,SAMPATIW,9999903825,2,2.0,5_10L,P,CAT,0,N,33.0,0.312963,1.331464,381.0,41.0,45.500000,2.500000,2,2


In [136]:
# we wont be taking num of loggins by referees, since its a direct depiction of how many people are activated
del df_model["num_referees_logging_in"]

In [137]:
test = pd.DataFrame()
for chumma in num_features:
    print(chumma, len(df_model[df_model[chumma] == "others"]))

months_since_last_login 0
months_since_last_trade 0
total_logins_one_year 0
total_trades_one_year 0
referees_avg_logins_in_last_year 0
referees_avg_trades_in_last_year 0


In [138]:
test

""


## categorizing all variables

In [139]:
df_test = df_model.copy()
cat_features = ["IncomePA", "cm_occup", "E2Channel", "IsKarvy", "KRA YN"]
num_features = ["months_since_last_login", "months_since_last_trade", "total_logins_one_year", 
               "total_trades_one_year", "referees_avg_logins_in_last_year","referees_avg_trades_in_last_year"]

 
df_model[num_features] = df_model[num_features].astype(float)

In [140]:
# function to categorize all features
def categorize(df):
    
    # taking only first 5 categories in all cat features
    for cat in  cat_features:
        df.loc[~df[cat].isin(df[cat].value_counts().index[:4]), cat] = "others"
    
    # divinding all numerical features into 5 equal quartiels
    for num in num_features:
        df[num] = pd.qcut(df[num], 7 ,duplicates='drop')

    
    return df
    

In [141]:
df_test = categorize(df_model.copy())

months_since_last_login
months_since_last_trade
total_logins_one_year
total_trades_one_year
referees_avg_logins_in_last_year
referees_avg_trades_in_last_year


In [156]:
# saving the data
df_test.to_csv("../inter_data/processed_categorized.csv")